# Basic Imports for Data Cleaning

In [12]:
import pandas as pd
pd.set_option('display.max_columns', None)

# Cleaning

In [19]:
csv_file_path = 'clean_merged_playerdata_with_weather.csv'

df = pd.read_csv(csv_file_path)

# drop columns we definitely don't need
df = df.drop(["bet_type", "tie_rule", "open_time", "close_time",

              'p1_player_name', 'p2_player_name', 'p3_player_name',
              "p1_outcome_text", "p2_outcome_text", "p3_outcome_text",
              "book", "event_completed", "event_name", "odds",
              'dg_id_p1', 'fin_text_p1',
              "course_num_p1",              
              'dg_id_p2', 'fin_text_p2',
              'dg_id_p3', 'fin_text_p3',
              'wx_lat', 'wx_lon', 'wx_date_from_close','wx_teetime',             
              'wx_datetime_hour','wx_conditions', 'wx_icon', 'wx_datetimeEpoch'], axis=1)

# rename columns we'd like to keep
df = df.rename(columns={'course_name_p1':'course_name'})
df = df.rename(columns={'tour_p1':'tour'})
df = df.rename(columns={'course_par_p1':'course_par'})


# dropping round scores
df = df.drop(["round_score_p1"], axis=1)
df = df.drop(["round_score_p2"], axis=1)
df = df.drop(["round_score_p3"], axis=1)


# preciptype can either only be nan or 'rain'
df['wx_preciptype'] = df['wx_preciptype'].fillna(0)
df['wx_preciptype'] = df['wx_preciptype'].apply(lambda x: 1 if x != 0 else x)


# drop rows that are missing player stats (player 1 stats should always be present no matter the match type for accurate analysis)
df.dropna(subset=['driving_dist_p1'], inplace=True)


# Create one outcome column
df['outcome'] = (
    df[['p1_outcome', 'p2_outcome', 'p3_outcome']]
    .fillna(0) # turn all na's or NaNs to 0
    .idxmax(axis=1) 
    .str.extract(r'p(\d+)_outcome') # pull out 1, 2, 3
    .astype(float) # convert to floats
)
# Then remove the other outcome columns
df = df.drop(['p1_outcome', 'p2_outcome', 'p3_outcome'], axis=1)
# drop rows with no outcome (shouldn't be any after previous cleaning, but just in case)
df.dropna(subset=['outcome'], inplace=True)


/var/folders/b1/w3z8y2710sg9wcwwsgn8f7sr0000gn/T/ipykernel_57201/3670626254.py:3: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file_path)


In [20]:
# create a csv of dataframe to inspect
df.to_csv('cleaned_golf_bets.csv', index=False)

In [21]:
csv_file_path_cleaned = 'cleaned_golf_bets.csv'
df = pd.read_csv(csv_file_path_cleaned)

print(df.head())

   p1_dg_id  p2_dg_id  p3_dg_id  p1_open  p1_close  p2_open  p2_close  \
0    6430.0   23604.0   27774.0      3.1       3.1      2.7       2.7   
1    6252.0   18474.0   21756.0      4.6       4.6      2.1       2.1   
2   20722.0   23504.0   25003.0      2.5       2.5      2.3       2.3   
3    6986.0    7960.0   12808.0      3.1       3.1      3.1       3.1   
4   19872.0   19960.0   24304.0      2.4       2.4      2.9       2.9   

   p3_open  p3_close  season  year  event_id  round_num tour      course_name  \
0     2.40      2.40    2025  2025       525        1.0  pga  TPC Twin Cities   
1     2.40      2.40    2025  2025       525        1.0  pga  TPC Twin Cities   
2     3.55      3.55    2025  2025       525        1.0  pga  TPC Twin Cities   
3     2.15      2.15    2025  2025       525        1.0  pga  TPC Twin Cities   
4     2.90      2.90    2025  2025       525        1.0  pga  TPC Twin Cities   

   course_par  start_hole_p1    teetime_p1  sg_putt_p1  sg_arg_p1  sg_app_